In [ ]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

from dataset.graph_builder import GraphBuilder

builder = GraphBuilder(dataset_paths=["data/MOLWENI/test.json"], dataset_names=["MOLWENI"], dataset_type='test')
builder()

In [ ]:
builder.display_graph(1)

In [ ]:
builder.save_graphs('data/MOLWENI/graphs')

In [ ]:
import os 

os.path.isdir('data/UNIFIED')

In [ ]:
import torch 
import os 
x = torch.load(os.path.join('data/MOLWENI/graphs/test', '556.pt'), map_location='cpu')
x 

In [ ]:
from utils import display_graph 
import torch 

display_graph(torch.load('data/BALANCED/graphs/test/212.pt', map_location='cpu'))